In [1]:
import re
import pandas
import random
import collections
import time
import csv

import requests
import bs4
import rpy2.interactive.packages

In [2]:
%load_ext rpy2.ipython

In [3]:
rpy2.interactive.packages.importr('rvest')

rpy2.robjects.packages.Package as a <module 'rvest'>

In [4]:
def get_dates(doi):
    base_url = 'http://dx.doi.org/'
    url = base_url + doi
    r_out = %R -i url url %>% rvest::html() %>% rvest::html_node('.articleinfo p:nth-child(3)') %>% rvest::html_text()
    received, accepted, published = r_out[0].split('; ')
    received, accepted, published = [str(pandas.to_datetime(x.split(':')[1].strip()).date()) for x in [received, accepted, published]]
    return {'doi': doi, 'received': received, 'accepted': accepted, 'published': published}

In [5]:
pubmed_df = pandas.read_table('data/pubmed.tsv.gz', compression='gzip')
pubmed_df = pubmed_df.dropna(subset=['doi'])
publication_types = collections.Counter(pubmed_df.pubtype)
pubmed_df = pubmed_df[pubmed_df.pubtype == 'Journal Article']

In [ ]:
def shuffle_and_head(df, k):
    return df.ix[random.sample(list(df.index), k)]

random.seed(0)
subset_df = pubmed_df.groupby('journal').apply(shuffle_and_head, k=1000)

In [ ]:
write_file = open('data/dates.tsv', 'w')
writer = csv.DictWriter(write_file,  delimiter='\t', fieldnames=['doi', 'received', 'accepted', 'published'])
writer.writeheader()
dois = list(subset_df.doi)
random.shuffle(dois)
for doi in dois:
    try:
        dates = get_dates(doi)
        writer.writerow(dates)
    except Exception as error:
        print(doi, error)
        pass
    time.sleep(11)
write_file.close()

10.1371/journal.pmed.0010016 need more than 1 value to unpack
